In [1]:
from saflow_params import *
from utils import get_SAflow_bids
import os
import mne
from neuro import find_logfile, remove_errors
import numpy as np
from autoreject import AutoReject
import pickle



In [ ]:
alldata = []
AR_logs = []
all_subjidx = []
all_oddball = []


for subj_idx, subj in enumerate(SUBJ_LIST):
    raws = []
    for run in BLOCS_LIST:
        
        # Open file
        bids_fpath = os.path.join(FOLDERPATH, 'sub-{}'.format(subj), 'ses-recording', 'meg', 'sub-{}_ses-recording_task-gradCPT_run-0{}_meg.ds'.format(subj, run))
        raw = mne.io.read_raw_ctf(bids_fpath, preload=True).resample(400)
        # Filter data
        high_cutoff = 150
        low_cutoff = 0.5
        raw.filter(low_cutoff, high_cutoff, fir_design="firwin")
        raw.notch_filter(np.arange(60, high_cutoff+1, 60), filter_length='auto',phase='zero', fir_design="firwin")
        # Get events
        try:
            events = mne.find_events(raw, min_duration=1/raw.info['sfreq'], verbose=False)
        except ValueError:
            events = mne.find_events(raw, min_duration=2/raw.info['sfreq'], verbose=False)
        log_file = LOGS_DIR + find_logfile(subj,run,os.listdir(LOGS_DIR))
        new_events,_,_ = remove_errors(log_file, events)
        # Segmentation
        tmin = 0
        tmax = 0.8
        baseline = None
        picks = mne.pick_types(raw.info, meg=True, eog=False, eeg=False, exclude='bads')
        epochs = mne.Epochs(raw, events=new_events, event_id=None, tmin=tmin,
                        tmax=tmax, baseline=baseline, reject=None, picks=picks, preload=True, event_repeated='merge')
        # Artifacts rejections
        ar = AutoReject(n_jobs=-2, verbose=False)
        epochs_clean, autoreject_log = ar.fit_transform(epochs, return_log=True)
        # Put each trial in the bigass file
        data = epochs_clean.get_data()
        for trial_idx, trial in enumerate(data):
            if new_events[trial_idx][2] == 21:### oddbal : freq=0, rare=1
                oddball = 0
            elif new_events[trial_idx][2] == 31:
                oddball = 1
            
            line = np.array([trial, subj_idx, oddball])
            alldata.append(line)
            
alldata = np.array(alldata)

alldata_path = os.path.join(FEAT_PATH, 'DLsaflow_alldata_autoreject.pickle')

data_dict = {'data':alldata, 
             'autoreject':AR_logs}

with open(alldata_path, 'wb') as fp:
    pickle.dump(data_dict, fp)

ds directory : /storage/Yann/saflow_DATA/saflow_bids/sub-04/ses-recording/meg/sub-04_ses-recording_task-gradCPT_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       1.74   79.24    0.00 mm <->    1.74   79.24   -0.00 mm (orig :  -67.36   48.31 -242.66 mm) diff =    0.000 mm
      -1.74  -79.24    0.00 mm <->   -1.74  -79.24    0.00 mm (orig :   43.50  -64.96 -245.83 mm) diff =    0.000 mm
     114.47    0.00    0.00 mm <->  114.46   -0.00   -0.00 mm (orig :   67.74   73.41 -252.87 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /storage/Yann/saflow_DATA/saflow_bids/sub-04/ses-recording/meg/sub-04_ses-recording_task-gradCPT_run-02_meg.ds/sub-04_ses-recording_task-gradCPT_run-02_meg.meg4: 
    System clock channel is available, c

<ipython-input-2-212e3af52547>:13: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw = mne.io.read_raw_ctf(bids_fpath, preload=True).resample(400)


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

529 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 529 events and 321 original time points ...
0 bad epochs dropped
Removing 5 compensators from info because not all compensation channels were picked.
99 coil definitions read
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Dropped 70 epochs
ds directory : /storage/Yann/saflow_DATA/saflow_bids

Current compensation grade : 3
Reading 0 ... 587999  =      0.000 ...   489.999 secs...
Trigger channel has a non-zero initial value of 10 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1452 events found
Event IDs: [21 31 99]
Trigger channel has a non-zero initial value of 10 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1452 events found
Event IDs: [21 31 99]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 150.00 Hz
- Upper transition bandwidth: 37.50 Hz (-6 dB cutof

<ipython-input-2-212e3af52547>:13: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw = mne.io.read_raw_ctf(bids_fpath, preload=True).resample(400)


...

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

437 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 437 events and 321 original time points ...
0 bad epochs dropped
Removing 5 compensators from info because not all compensation channels were picked.
99 coil definitions read
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Dropped 172 epochs
ds directory : /storage/Yann/saflow_DATA/saflow_bid

<ipython-input-2-212e3af52547>:13: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw = mne.io.read_raw_ctf(bids_fpath, preload=True).resample(400)


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

446 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 446 events and 321 original time points ...
0 bad epochs dropped
Removing 5 compensators from info because not all compensation channels were picked.
99 coil definitions read
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Dropped 165 epochs
ds directory : /storage/Yann/saflow_DATA/saflow_bid

<ipython-input-2-212e3af52547>:13: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw = mne.io.read_raw_ctf(bids_fpath, preload=True).resample(400)


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

431 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 431 events and 321 original time points ...
0 bad epochs dropped
Removing 5 compensators from info because not all compensation channels were picked.
99 coil definitions read
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Removing 5 compensators from info because not all compensation channels were picked.
Dropped 144 epochs
ds directory : /storage/Yann/saflow_DATA/saflow_bid

In [ ]:
new_raw = mne.concatenate_epochs(raws)

In [ ]:
raw_f = mne.preprocessing.maxwell_filter(raw)

In [ ]:
epochs[2].info['dev_head_t']

In [ ]:

with open(alldata_path, 'wb') as fp:
    pickle.dump(data_dict, fp)